# Лабораторная работа №3. Методы многомерной оптимизации с ограничениями

Цель работы: ознакомление с методами поиска минимума функции двух переменных в оптимизационных задачах c ограничениями (метод множителей Лагранжа, метод Зойтендейка, метод проектируемого градиента Д. Розена).

$f(x)=x_1^2-x_2^2$

$𝜑(x)=x_1-2x_2-5=0$

In [1]:
import numpy as np
from scipy.optimize import minimize, minimize_scalar, linprog, Bounds, LinearConstraint

In [2]:
# сама функция
def f(x):
  return x[0]**2 - x[1]**2

In [3]:
def grad_f(x):
  return np.array([2*x[0], -2*x[1]], dtype='f')

In [4]:
# ограничение
def g(x):
  return x[0] - 2*x[1] - 5

### Метод множителей Лагранжа

In [5]:
def lagrange(x,lm):
  return f(x) + lm * g(x)

In [6]:
def grad_lagrange(x,lm):
  grad = np.array([2*x[0],2*x[1]])
  grad_constr = np.array([1,-2])

  return grad + lm*grad_constr

In [7]:
x0 = [17,6]
lm0 = 0

In [8]:
result = minimize(lagrange, x0, args=(lm0,), method='SLSQP', jac=grad_lagrange,
                  bounds=Bounds(0, float('inf')), 
                  constraints=LinearConstraint(np.array([1,-2]), 5, 5))

In [9]:
result

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 24.99999999999492
       x: [ 5.000e+00  0.000e+00]
     nit: 2
     jac: [ 1.000e+01  0.000e+00]
    nfev: 2
    njev: 2

In [10]:
print("Оптимальная точка:", result.x)
print("Значение функции в опт.точке:", result.fun)

Оптимальная точка: [5. 0.]
Значение функции в опт.точке: 24.99999999999492


### Метод Зойтендейка

In [11]:
def zoutendjik(x0, eps=0.01, max_iters=np.inf):
  x = x0
  iters = 0
  while True:
    iters += 1
    # решаем задачу P1
    c = grad_f(x)
    A_ub = [] # инициализируем пустую матрицу активных ограничений
    b_ub = [] # инициализируем пустой вектор активных ограничений
    
    # если x1 = 0, то x1 - активное ограничение, добавляем в список
    if abs(x[0]) <= eps: 
      A_ub.append([-1, 0])
      b_ub.append(0)
    # если x2 = 0, то x2 - активное ограничение, добавляем в список
    if abs(x[1]) <= eps: 
      A_ub.append([0, -1])
      b_ub.append(0)
    # если активных ограничений нет:
    if not len(A_ub): 
      A_ub = None
      b_ub = None
    # при условии Hs = 0
    A_eq = np.array([[1, -2]])
    b_eq = [0]
    bounds = [(-1, 1)]*2

    # находим возможное направление симплекс-методом
    result = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds)

    # возвращаем точку Куна-Таккера, если значение целевой функции в задаче P1 = 0
    if np.abs(result.fun) <= eps: 
      return x

    # процедура поиска максимального alpha
    b_tilda = np.array([0, 0]) - np.matmul(np.array([[-1,0],[0,-1]]), x)
    s_tilda = np.matmul(np.array([[-1,0],[0,-1]]),result.x)

    alphas = []
    for i in range(2):
      if s_tilda[i] > 0:
        alphas.append(b_tilda[i] / s_tilda[i])

    alpha_max = min(alphas) if alphas else np.inf

    # находим оптимальную alpha
    alpha = minimize_scalar(lambda alpha: f(x+alpha*result.x), bounds=(0, alpha_max)).x
    x = x + alpha * result.x # переход к следующей итерации

    if iters >= max_iters: return x

In [12]:
result = zoutendjik((17,6))
print("Оптимальная точка:", result)
print("Значение функции:", f(result))

Оптимальная точка: [5.00000645e+00 3.22494300e-06]
Значение функции: 25.00006449889118


### Метод проектируемого градиента Д.Розена

$P = I - M^T(MM^T)^{-1}M$

$w = -(MM^T)^{-1}M\nabla f(x) = \begin{pmatrix} u \\ v \end{pmatrix}$

In [13]:
def rosen(x0, eps=0.01, max_iters=np.inf):
  x = x0
  A = [] # инициализируем пустую матрицу активных ограничений
  b = [] # инициализируем пустой вектор активных ограничений
    
  # если x1 = 0, то x1 - активное ограничение, добавляем в список
  if abs(x[0]) <= eps: 
    A.append([-1, 0])
    b.append(0)
  # если x2 = 0, то x2 - активное ограничение, добавляем в список
  if abs(x[1]) <= eps: 
    A.append([0, -1])
    b.append(0)
  if not len(A): 
    A = None
    b = None

  # ограничение-равенство
  H = np.array([[1, -2]])
  d = [5]

  iters = 0
  while True:
    iters += 1

    # если активных ограничений нет
    if A is None:
      M = np.array(H)
    else:
      M = np.vstack((A, H))
    
    Q = np.dot(np.dot(M.T, np.linalg.inv(np.dot(M, M.T))), M)
    P = np.identity(2) - Q

    # вычисляем возможное направление спуска
    s = -np.dot(P, grad_f(x))

    # если значение равно нулю
    if np.allclose(s, [0,0]):
      if not len(M): return x # если матрица пуста, останавливаемся

      # если нет, ищем вектор w
      w = -np.dot(np.dot(np.linalg.inv(np.dot(M, M.T)), M), grad_f(x))

      if len(A): # если есть активные ограничения
        nums = len(w) - len(A)
        for i in range(nums): # ищем отрицательные значения
          if w[i] < 0: 
            A = np.delete(A, i, axis=0)
            break
        else: return x # если таковых нет, останавливаемся
        continue

    # далее, процедура нахождения максимального альфа
    b_tilda = np.array([0, 0]) - np.matmul(np.array([[-1,0],[0,-1]]),x)
    s_tilda = np.matmul(np.array([[-1,0],[0,-1]]), s)

    alphas = []
    for i in range(2):
      if s_tilda[i] > 0:
        alphas.append(b_tilda[i] / s_tilda[i])

    alpha_max = min(alphas) if alphas else np.inf

    alpha = minimize_scalar(lambda alpha: f(x+alpha*s), bounds=(0, alpha_max)).x
    x = x + alpha * s

    if A is None: A = []
    if b is None: b = []
    # если x1 = 0, то x1 - активное ограничение, добавляем в список
    if abs(x[0]) <= eps: 
      A.append([-1, 0])
      b.append(0)
    # если x2 = 0, то x2 - активное ограничение, добавляем в список
    if abs(x[1]) <= eps: 
      A.append([0, -1])
      b.append(0)
    if not len(A): 
      A = None
      b = None

    if iters >= max_iters: return x

In [14]:
result = rosen((17,6))
print("Оптимальная точка:", result)
print("Значение функции:", f(result))

Оптимальная точка: [5.00011242e+00 5.62120937e-05]
Значение функции: 25.001124251352667
